In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
#choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
#frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


In [12]:
offertimes = [20,30,40,50,60,75,100]
offertime_lst = [offer_times_20,offer_times_30,offer_times_40,offer_times_50,offer_times_60,offer_times_75,offer_times_100]
choice_collection_lst = [choice_collection_20,choice_collection_30,choice_collection_40,choice_collection_50,choice_collection_60,choice_collection_75,choice_collection_100]

In [13]:
lcmnl_kl_lst = []
for k in range(len(offertimes)):
    df_lcmnl_prob = pd.read_csv('limit/probability/lcmnl_em/prob_offertimes'+str(offertimes[k])+'.csv')
    df_lcmnl_prob = df_lcmnl_prob.drop(df_lcmnl_prob.columns[0],axis=1)
    lcmnl_prob = np.array(df_lcmnl_prob)
    lcmnl_kl = 0 
    for i in range(lcmnl_prob.shape[0]):
        for j in range(lcmnl_prob.shape[1]):
            if lcmnl_prob[i][j] >0:
                lcmnl_kl = lcmnl_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(lcmnl_prob[i][j])/np.sum(offertime_lst[k])
    lcmnl_kl_lst.append(lcmnl_kl)

In [14]:
lcmnl_kl_lst

[0.5856461268848074,
 0.5824346005751264,
 0.5784750511347461,
 0.5773594592095833,
 0.5748309570377605,
 0.5731514699887693,
 0.572776981343711]

In [15]:
mnl_kl_lst = []
for k in range(len(offertimes)):
    df_mnl_prob = pd.read_csv('limit/probability/mnl/prob_offertimes'+str(offertimes[k])+'.csv')
    df_mnl_prob = df_mnl_prob.drop(df_mnl_prob.columns[0],axis=1)
    mnl_prob = np.array(df_mnl_prob)
    mnl_kl = 0 
    for i in range(mnl_prob.shape[0]):
        for j in range(mnl_prob.shape[1]):
            if mnl_prob[i][j] >0:
                mnl_kl = mnl_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(mnl_prob[i][j])/np.sum(offertime_lst[k])
    mnl_kl_lst.append(mnl_kl)

In [16]:
mnl_kl_lst

[0.5937364984479344,
 0.5896719056441664,
 0.585088075111838,
 0.5822274721946021,
 0.5789814998084744,
 0.5772794827261699,
 0.576775281965606]

In [17]:
mdm_kl_lst = []
for k in range(len(offertimes)):
    df_mdm_prob = pd.read_csv('limit/probability/mdm/prob_offertimes'+str(offertimes[k])+'.csv')
    df_mdm_prob = df_mdm_prob.drop(df_mdm_prob.columns[0],axis=1)
    mdm_prob = np.array(df_mdm_prob)
    mdm_kl = 0 
    for i in range(mdm_prob.shape[0]):
        for j in range(mdm_prob.shape[1]):
            if mdm_prob[i][j] >0:
                mdm_kl = mdm_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(mdm_prob[i][j])/np.sum(offertime_lst[k])
    mdm_kl_lst.append(mdm_kl)

In [18]:
kl_df = pd.DataFrame({'mdm':mdm_kl_lst,'lcmnl':lcmnl_kl_lst,'mnl':mnl_kl_lst})
kl_df

,mdm,lcmnl,mnl
0,0.587444,0.585646,0.593736
1,0.583676,0.582435,0.589672
2,0.579726,0.578475,0.585088
3,0.578266,0.577359,0.582227
4,0.577417,0.574831,0.578981
5,0.573529,0.573151,0.577279
6,0.573159,0.572777,0.576775


In [19]:
dt_20 = 0
for i in range(choice_collection_20.shape[0]):
    for j in range(choice_collection_20.shape[1]):
        if choice_collection_20[i][j] >0:
            dt_20 = dt_20 + offer_times_20[j]*choice_collection_20[i][j]*np.log(choice_collection_20[i][j])/np.sum(offer_times_20)

dt_30 = 0
for i in range(choice_collection_30.shape[0]):
    for j in range(choice_collection_30.shape[1]):
        if choice_collection_30[i][j] >0:
            dt_30 = dt_30 + offer_times_30[j]*choice_collection_30[i][j]*np.log(choice_collection_30[i][j])/np.sum(offer_times_30)

dt_40 = 0
for i in range(choice_collection_40.shape[0]):
    for j in range(choice_collection_40.shape[1]):
        if choice_collection_40[i][j] >0:
            dt_40 = dt_40 + offer_times_40[j]*choice_collection_40[i][j]*np.log(choice_collection_40[i][j])/np.sum(offer_times_40)
dt_50 = 0
for i in range(choice_collection_50.shape[0]):
    for j in range(choice_collection_50.shape[1]):
        if choice_collection_50[i][j] >0:
            dt_50 = dt_50 + offer_times_50[j]*choice_collection_50[i][j]*np.log(choice_collection_50[i][j])/np.sum(offer_times_50)

dt_60 = 0
for i in range(choice_collection_60.shape[0]):
    for j in range(choice_collection_60.shape[1]):
        if choice_collection_60[i][j] >0:
            dt_60 = dt_60 + offer_times_60[j]*choice_collection_60[i][j]*np.log(choice_collection_60[i][j])/np.sum(offer_times_60)


dt_75 = 0
for i in range(choice_collection_75.shape[0]):
    for j in range(choice_collection_75.shape[1]):
        if choice_collection_75[i][j] >0:
            dt_75 = dt_75 + offer_times_75[j]*choice_collection_75[i][j]*np.log(choice_collection_75[i][j])/np.sum(offer_times_75)


dt_100 = 0
for i in range(choice_collection_100.shape[0]):
    for j in range(choice_collection_100.shape[1]):
        if choice_collection_100[i][j] >0:
            dt_100 = dt_100 + offer_times_100[j]*choice_collection_100[i][j]*np.log(choice_collection_100[i][j])/np.sum(offer_times_100)

deterministic_terms_lst = [dt_20,dt_30,dt_40,dt_50,dt_60,dt_75,dt_100]

In [20]:
mdm_kl_loss_lst = []
lcmnl_kl_loss_lst = []
mnl_kl_loss_lst = []
for i in range(len(offertimes)):
    mdm_kl_loss_lst.append(mdm_kl_lst[i]+deterministic_terms_lst[i])
    lcmnl_kl_loss_lst.append(lcmnl_kl_lst[i]+deterministic_terms_lst[i])
    mnl_kl_loss_lst.append(mnl_kl_lst[i]+deterministic_terms_lst[i])
kl_loss_df = pd.DataFrame({'offertimes':offertimes,'assortmentsize':[len(assortments_20),len(assortments_30),len(assortments_40),len(assortments_50),len(assortments_60),len(assortments_75),len(assortments_100)],
                           'mdm':mdm_kl_loss_lst,'lcmnl':lcmnl_kl_loss_lst,'mnl':mnl_kl_loss_lst})
kl_loss_df

,offertimes,assortmentsize,mdm,lcmnl,mnl
0,20,29,0.003563,0.001765,0.009855
1,30,24,0.002848,0.001607,0.008844
2,40,19,0.002753,0.001502,0.008115
3,50,15,0.002065,0.001159,0.006027
4,60,13,0.003517,0.000931,0.005082
5,75,12,0.001314,0.000937,0.005065
6,100,11,0.001329,0.000948,0.004946


In [22]:
kl_loss_df.to_csv('limit/kl_directly_using1norm_solution.csv')
kl_loss_df